In [1]:
import pandas as pd
import os
import re
import codecs
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

In [56]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Acquire

In [2]:
filename = 'book30-listing-train.csv'

In [3]:
header_names = ['Amazon ID (ASIN)', 'Filename', 'Image URL', 'Title', 'Author', 'Category ID',
                'Category']
with codecs.open(filename, mode='r', encoding='utf-8', errors='ignore') as f:
    df = pd.read_csv(f, delimiter=",", header=None, names=header_names)

# Subset Dataframe

In [4]:
df = df[['Author','Title','Category']]

In [5]:
keeper_columns = ['Romance','Mystery, Thrillers & Suspense',
                  'Teen & Young Adult','Science Fiction & Fantasy',
                  'Literature & Fiction','Humor & Entertainment',
                  ]

In [6]:
df = df[df['Category'].isin(keeper_columns)]

# Nulls

In [7]:
df = df.dropna()

In [8]:
df = df.reset_index(drop=True)

In [9]:
def clean_column(df, column):
    # Convert column to lowercase and remove text inside parentheses
    df['temp'] = df[column].str.lower().replace(r'\([^()]*\)', '', regex=True)

    # Remove text after colon or hyphen
    df['temp'] = df['temp'].str.split(r'[:\-]').str[0]

    # Remove extra whitespace
    df['temp'] = df['temp'].str.replace(r'\s+', ' ', regex=True).str.strip()

    # Modify DataFrame with new column
    new_column = f'cleaned_{column}'
    df[new_column] = df['temp']

    # Remove temporary column
    df = df.drop(columns=['temp'])

    return df

# Clean

In [10]:
df = clean_column(df,'Title')

In [11]:
df = clean_column(df,'Author')

In [12]:
df.sample(5)

,Author,Title,Category,cleaned_Title,cleaned_Author
3215,Dallas Jessup,Young Revolutionaries Who Rock: An Insider's G...,Teen & Young Adult,young revolutionaries who rock,dallas jessup
3256,Ted Hughes,Crow: From the Life and Songs of the Crow,Literature & Fiction,crow,ted hughes
5563,George Lucas,Star Wars Trilogy,Science Fiction & Fantasy,star wars trilogy,george lucas
6356,Marilyn Nelson,A Wreath for Emmett Till (Boston Globe-Horn Bo...,Teen & Young Adult,a wreath for emmett till (boston globe,marilyn nelson
2587,Amy Maltman,Ride Every Stride,Literature & Fiction,ride every stride,amy maltman


In [13]:
title = df['cleaned_Title']
author = df['cleaned_Author']

# Selenium

In [70]:
driver = webdriver.Chrome()
driver.get("https://www.goodreads.com/book/show/3450744-nudge")

In [71]:
df['Title'].iloc[0]

'Breaking News (Godmothers, Book 5) (The Godmothers)'

In [72]:
df['cleaned_Title'].iloc[0]

'breaking news'

In [73]:
search_bar = driver.find_element_by_xpath('//*[@id="Header"]/div[2]/div[2]/section/form/input[1]')

### Clicked Search

In [74]:
search_bar.send_keys(title.iloc[0] + " " + author.iloc[0])
search_bar.submit()

In [75]:
links = driver.find_element_by_class_name('bookTitle')

In [78]:
links.get_attribute('href')

'https://www.goodreads.com/book/show/13771831-breaking-news?from_search=true&from_srp=true&qid=4COxo43vyt&rank=1'

# Raw HTML?

In [32]:
html_content = driver.page_source

# Beautiful Soup

In [46]:
soup = BeautifulSoup(html_content, 'html.parser')

In [52]:
link_tag = soup.find('a', class_='bookTitle')

In [51]:
link_tag

In [48]:
link = link_tag.get('href')

AttributeError: 'NoneType' object has no attribute 'get'